In [4]:
!pip install folium==0.5.0
! pip install geocoder
! pip install geopy

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

print("Libraries imported.")

Libraries imported.


In [5]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create a list to store neighborhood data
neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head(50)

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan
5,Bandar Tun Razak
6,Bangsar
7,Bangsar Park
8,Bangsar South
9,Batu 11 Cheras


In [6]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [7]:
coords = [get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist()]
coords

[[3.0576900000000364, 101.74388000000005],
 [3.1484988508598852, 101.69672774991264],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.0827600000000643, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1486092228953293, 101.69985445868105],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.1364442625375997, 101.69029430612223],
 [3.1578250453611005, 101.69728006780952],
 [3.083300000000065, 10

In [8]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082760,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


In [9]:
kl_df.to_csv("kl_df.csv", index=False)